In [1]:
# Import all packages used
import pandas as pd
import numpy as np
import spacy as sp
import string
import en_core_web_sm
import nltk as nl
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import svm

from sklearn.ensemble import RandomForestClassifier


from sklearn.naive_bayes import BernoulliNB

model = svm.SVC()
param = {'kernel': ['linear','rbf']}

lemm_obj = sp.load('en_core_web_lg')
lemm_obj.max_length = 1500000
ps = nl.PorterStemmer()

vect_cnt = CountVectorizer(analyzer='word',min_df=1,stop_words='english',token_pattern='[a-zA-Z0-9]{1,}')
tfidf_vect = TfidfVectorizer(analyzer = 'word', min_df = 1, stop_words = 'english',vocabulary=20000, token_pattern = '[a-zA-Z0-9]{1,}')

allowed_postags = ['NOUN','ADJ','VERB','ADV']

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import IPython

C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
gen_df_encode = pd.read_pickle("D:\Machine Learning\Vamsi Thesis\Code\Results\gen_df_encode.pkl")

In [3]:
fullMovieData = pd.read_pickle(r"D:\Machine Learning\Vamsi Thesis\Code\Results\fullMovieData.pkl")
# List of genres
g_list = fullMovieData.apply(lambda x: x['genres'].split('|'),axis = 1)
g_list=g_list.to_list()
g_list=list(set(sum(g_list,[])))
g_list

['War',
 'Action',
 'Drama',
 'Horror',
 'Mystery',
 'IMAX',
 'Crime',
 'Western',
 'Animation',
 'Comedy',
 'Romance',
 'Thriller',
 'Film-Noir',
 'Fantasy',
 'Documentary',
 'Sci-Fi',
 'Adventure',
 'Musical',
 'Children']

In [4]:
Final_data=pd.DataFrame(columns=["Genre","Actual Positives","Actual Negatives","Vectorizer","Model","Test_Precision","Test_Recall","Test_Fscore","Precision","Recall","Fscore"])
Final_data.shape
#Master function SVM RF Naive Bayes
def prediction(df,text, genre, vectorizer, model, param):
    result=[]
    print("Genre for prediction: ", genre)
    result.append(genre)
    
    #Sampling
    df_model_0 = df[df[genre] == 0]
    df_model_1 = df[df[genre] == 1]
    print("Number of 0s: ", df_model_0.shape[0])
    print("Number of 1s: ", df_model_1.shape[0])
    result.append(df_model_0.shape[0])
    result.append(df_model_1.shape[0]) 
    
    sample_size = 2*df_model_1.shape[0]
    if sample_size > len(df_model_0) : sample_size = len(df_model_0)
    df_model_s=pd.concat([df_model_1,df_model_0.sample(sample_size)])
    print("Shape of the Sampled DataFrame: ", df_model_s.shape)
    #result.append(df_model_s.shape[0])
    
    #Train-Test Split
    X_df = df_model_s[text]
    y_df = df_model_s[genre]
    X_train,X_test,y_train,y_test = train_test_split(X_df, y_df, test_size = 0.25, random_state = 1,
                                                     shuffle = True)
    
    #Vectorization
    print("\n")
    print("Vectorization Start")
    vect = vectorizer(analyzer = 'word', min_df = 1, stop_words = 'english', token_pattern = '[a-zA-Z]{2,}')
    vect_fit = vect.fit(X_train)
    result.append(vectorizer)
    
    X_train_vec = pd.DataFrame(vect_fit.transform(X_train).toarray())
    X_train_vec.columns = vect_fit.get_feature_names()
    
    X_test_vec = pd.DataFrame(vect_fit.transform(X_test).toarray())
    X_test_vec.columns = vect_fit.get_feature_names()
    print("Vectorization End")
    
    #Model
    print("\n")
    print("Model Building Start")
    model = model
    param = param
    result.append(model)
    
    gs = GridSearchCV(model, param, cv = 5,n_jobs=2)
    gs_fit = gs.fit(X_train_vec, y_train)
    
    test_pred_cnt = pd.DataFrame(gs.predict(X_test_vec))
    train_pred_cnt = pd.DataFrame(gs.predict(X_train_vec))
    print("Model Building End")

    # Accuracy metrics

    #Test Accuracy
    print("\n")
    print("Sample data Test Set Confusion Matrix: ")
    print(confusion_matrix(y_test, test_pred_cnt))
    precision, recall, fscore, train_support = score(y_test, test_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Test Set Precision: ", precision)
    print("Sample data Test Set Recall: ", recall)
    print("Sample data Test Set FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])

    # Train Accuracy
    print("Sample data Train Set Confusion Matrix: ")
    print(confusion_matrix(y_train, train_pred_cnt))
    precision, recall, fscore, train_support = score(y_train, train_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Train Set (Precision, Recall, Fscore) :",precision, recall, fscore)
    print("\n")

    #Accuracy on full Data
    X_df_full = df[text]
    y_df_full = df[genre]

    # vectorizer
    X_full_tf = pd.DataFrame(vect_fit.transform(X_df_full).toarray())

    print("Probability Evaluation and Concat Start")
    full_pred_cnt = pd.DataFrame(gs.predict(X_full_tf))
    full_pred_prob = pd.DataFrame(gs.predict_proba(X_full_tf))
    df = pd.concat([df,full_pred_prob], axis = 1)
    df.rename(columns ={1:genre+'_prob'},inplace=True)
    df.drop([0],axis=1,inplace=True)
    print("Probability Evaluation and Concat End")
    
    print("\n")
    print("Full Dataset Confusion Matrix " , model)
    print(confusion_matrix(y_df_full, full_pred_cnt))
    precision, recall, fscore, train_support = score(y_df_full, full_pred_cnt, pos_label = 1, average = 'binary')
    print("Full Dataset Precision: ", precision)
    print("Full Dataset Recall: ", recall)
    print("Full Dataset FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])
    
    print("Appending results to Final_data Start")
    Final_data.loc[len(Final_data)]=result
    print("Appending results to Final_data End")
    return df

In [5]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test= prediction(df_test,'plot_lemm',i,TfidfVectorizer, RandomForestClassifier(), {'n_estimators': [10,25,50,100],'max_depth':[100]})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\RF_Probability_Distribution_plots.pkl')
Final_data.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_RF.pkl')
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  War
Number of 0s:  10100
Number of 1s:  302
Shape of the Sampled DataFrame:  (906, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[140  11]
 [ 23  53]]
Sample data Test Set Precision:  0.828125
Sample data Test Set Recall:  0.6973684210526315
Sample data Test Set FScore:  0.7571428571428572


Sample data Train Set Confusion Matrix: 
[[453   0]
 [  0 226]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 1.0 1.0


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[9636  464]
 [  23  279]]
Full Dataset Precision:  0.3755047106325707
Full Dataset Recall:  0.9238410596026491
Full Dataset FScore:  0.5339712918660288


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Action
Number of 0s:  8933
Number of 1s:  1469
Shape of the Sampled DataFrame:  (44

Full Dataset Precision:  0.37840785169029445
Full Dataset Recall:  0.8589108910891089
Full Dataset FScore:  0.5253595760787282


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Comedy
Number of 0s:  6914
Number of 1s:  3488
Shape of the Sampled DataFrame:  (10402, 38)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[1687   28]
 [ 751  135]]
Sample data Test Set Precision:  0.8282208588957055
Sample data Test Set Recall:  0.1523702031602709
Sample data Test Set FScore:  0.25738798856053385


Sample data Train Set Confusion Matrix: 
[[5199    0]
 [ 222 2380]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.9146810146041506 0.9554395824969891


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[6886   28]
 [ 973 2515]]
Full Dataset Precision:  0.988989382618954
Full Dat

C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sample data Test Set Precision:  0.0
Sample data Test Set Recall:  0.0
Sample data Test Set FScore:  0.0


Sample data Train Set Confusion Matrix: 
[[37  0]
 [ 0 26]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 1.0 1.0


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[10320    54]
 [    2    26]]
Full Dataset Precision:  0.325
Full Dataset Recall:  0.9285714285714286
Full Dataset FScore:  0.48148148148148157


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Fantasy
Number of 0s:  9813
Number of 1s:  589
Shape of the Sampled DataFrame:  (1767, 42)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[293  13]
 [ 81  55]]
Sample data Test Set Precision:  0.8088235294117647
Sample data Test Set Recall:  0.40441176470588236
Sample data Test Set FScore:  0.539215686274509

NameError: name 'end' is not defined

In [6]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test= prediction(df_test,'tt_lemm',i,TfidfVectorizer, RandomForestClassifier(), {'n_estimators': [10,25,50,100],'max_depth':[100]})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\RF_Probability_Distribution_tt.pkl')
Final_data.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_RF.pkl')
end = time.time()
total_time = end-start


print("Total Time taken:" + str(end-start))

Genre for prediction:  War
Number of 0s:  10100
Number of 1s:  302
Shape of the Sampled DataFrame:  (906, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[144   7]
 [ 47  29]]
Sample data Test Set Precision:  0.8055555555555556
Sample data Test Set Recall:  0.3815789473684211
Sample data Test Set FScore:  0.5178571428571429


Sample data Train Set Confusion Matrix: 
[[452   1]
 [  2 224]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9955555555555555 0.9911504424778761 0.9933481152993349


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[9620  480]
 [  49  253]]
Full Dataset Precision:  0.3451568894952251
Full Dataset Recall:  0.8377483443708609
Full Dataset FScore:  0.4888888888888889


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Action
Number of 0s:  8933
N

 [  73  331]]
Full Dataset Precision:  0.5697074010327022
Full Dataset Recall:  0.8193069306930693
Full Dataset FScore:  0.6720812182741117


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Comedy
Number of 0s:  6914
Number of 1s:  3488
Shape of the Sampled DataFrame:  (10402, 38)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[1609  106]
 [ 734  152]]
Sample data Test Set Precision:  0.5891472868217055
Sample data Test Set Recall:  0.17155756207674944
Sample data Test Set FScore:  0.26573426573426573


Sample data Train Set Confusion Matrix: 
[[5156   43]
 [ 263 2339]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9819479429051218 0.898923904688701 0.9386035313001605


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[6765  149]
 [ 997 2491]]
Full Dataset Precision:

Sample data Test Set FScore:  0.4423076923076923


Sample data Train Set Confusion Matrix: 
[[428   0]
 [  2 209]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.990521327014218 0.9952380952380951


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[9602  516]
 [  52  232]]
Full Dataset Precision:  0.31016042780748665
Full Dataset Recall:  0.8169014084507042
Full Dataset FScore:  0.44961240310077516


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Children
Number of 0s:  9884
Number of 1s:  518
Shape of the Sampled DataFrame:  (1554, 47)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[263   6]
 [ 73  47]]
Sample data Test Set Precision:  0.8867924528301887
Sample data Test Set Recall:  0.39166666666666666
Sample data Test Set FScore:  0.5433526011560694


Sample d